In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
import time 
import numpy as np
import matplotlib.pyplot as plt 
plt.rcParams['figure.figsize'] = 10, 10
from utilities import *
import math
import pandas as pd
from itertools import combinations

In [3]:
# read the data 
def process_inhval(*args):
    for val in args: 
        if val == 'NA':
            yield 0
        else:
            yield float(val)
readfilename = 'PI_DataSet_6_19_all.txt'
r = open(readfilename)
header = process_line(next(r))

In [4]:
# things to change 
vectype = 'distvec'
numinhs = 2 

In [5]:
inh_start = header.index('FPV') 
inh_end = header.index('DRV')+1

vec_start = header.index('dist_vec_start') if vectype =='distvec' else header.index('count_vec_start')
vec_end = header.index('count_vec_start') if vectype =='distvec' else len(header)
vals = []
for line in r:
    if line!= '\n':
        line = process_line(line)
        token = list(process_inhval(*line[inh_start:inh_end])) + [float(x) for x in line[vec_start:vec_end]]
        vals.append(token)
r.close()
vals = np.array(vals)
ind_inh = {v:i for i,v in enumerate(header[inh_start:inh_end])}
def filter_inhs(data, ind_dict, *args):
    vec_length = 210
    names = sorted([(ind_dict[name],name) for i,name in enumerate(args)])
    inds = list(map(lambda x:x[0], names))
    names = list(map (lambda x:x[1], names))
    tmp = np.hstack((data[:,inds],data[:,8:]))
    df = pd.DataFrame(tmp, columns = ['c'+str(i) for i in range(len(tmp[0]))])
    query_string = ' & '.join(['c'+str(i)+">0" for i in range(len(inds))])
    df = df.query(query_string)
    for i,name in enumerate(names):
        df[names[i]] = np.where(df['c'+str(i)] > 3.0 , '1', '0')
    df['target'] = df[names].apply(lambda x:int(''.join(x),2),axis = 1)
    return df.iloc[:,len(names):(-len(names)-1)].values,df['target'].values.reshape(-1,1)

In [7]:
header = ["inh"+str(i+1) for i in range(numinhs)]+["test_accuracy","test_precision","test_recall","test_f1","avg_3fold_acc","time"]
results = pd.DataFrame(columns=header)
ind = 0 
inhs = ["ATV","DRV","FPV","IDV","LPV","NFV","SQV","TPV"]
combs = list(combinations(inhs,numinhs))
t0 = time.time()
for tup in combs:
    print(tup)
    X,y = filter_inhs(vals,ind_inh,*tup)
    max_ = X.max()
    X = X/max_
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    model = RandomForestClassifier(n_estimators=2, bootstrap = False,max_features = 210, verbose = 1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_pred,y_test)*100
    
    print("test_acc:",printfour(acc))
    prf_score = precision_recall_fscore_support(y_test,y_pred,average='weighted')[:-1]
    prf_score = [printfour(s*100) for s in prf_score]
    print(prf_score)
    scores = cross_val_score(model, X_train, y_train, cv=3)
    mean_scores = np.mean(scores*100)
    scores = [printfour(s*100) for s in scores]
    print("cross val accuracy:", scores)
    print("average cross val accuracy:",printfour(mean_scores))
    tt = printfour(time.time()-t0)
    print("time taken:",tt)
    print("______________")
    row = list(tup) + [acc] + prf_score + [mean_scores,tt]
    results.loc[ind] = row
    ind += 1

('ATV', 'DRV')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   37.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


test_acc: 99.8904
['99.8904', '99.8904', '99.8903']


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   26.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   25.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   24.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished


cross val accuracy: ['99.8928', '99.8753', '99.849']
average cross val accuracy: 99.8723
time taken: 116.5453
______________
('ATV', 'FPV')


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   33.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished


test_acc: 99.8642
['99.864', '99.8642', '99.864']


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   25.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   24.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


cross val accuracy: ['99.8267', '99.8136', '99.7993']
average cross val accuracy: 99.8132
time taken: 5233.2053
______________


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   25.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished


In [8]:
results.round(4).to_csv('distvec_two_inhibitors_randomforest_aprf_avg3fold.csv')